TODO : generaliser à images 32x32, etc.

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
#from waveimage import WaveImage, calc_dim, calc_U, mnist_reshape_32
from waveimage import WaveImage, calc_dim, calc_U, mnist_reshape_32

In [4]:
from scipy.stats import multivariate_normal, entropy

In [5]:
import math

In [6]:
import sys, os

In [7]:
import pickle

In [8]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
from waveimage import calc_U

## Creation de la base d'apprentissage

In [10]:
def wave_tensor_data(batch_x):
    batch_size, _ = batch_x.shape
    wave_tensor = {}
    for h in range(6):
        if h == 0:
            h_size = 1
            wave_tensor[h] = np.zeros((batch_size, h_size, h_size, 1))
        else:
            h_size = 2**(h - 1)
            wave_tensor[h] = np.zeros((batch_size, h_size, h_size, 3))
    for num_batch in range(batch_size):
        image = mnist_reshape_32(batch_x[num_batch])
        w = WaveImage(image = image)
        for h in range(w.get_h_max()):
            data_h = w.get_data()[h]
            if h == 0:
                wave_tensor[h][num_batch][0][0][0] = data_h[(0,0)]
            else:
                for u in data_h:
                    wave_tensor[h][num_batch][u[0]][u[1]][:] = data_h[u]
    return wave_tensor

In [11]:
def calc_pow2(i_ref):
    pow2_i = np.zeros(5, dtype='int')
    reste = i_ref
    for p in range(4,-1,-1):
        pow2_i[p] = int(reste // 2**p)
        #reste = reste % 2**p
    return pow2_i[::-1]


In [12]:
print calc_pow2(15)

[ 0  1  3  7 15]


In [13]:
def init_wave_tensor(batch_size):
    wave_tensor = {}
    for h in range(6):
        if h == 0:
            h_size = 1
            wave_tensor[h] = np.zeros((batch_size, h_size, h_size, 1))
        else:
            h_size = 2**(h - 1)
            wave_tensor[h] = np.zeros((batch_size, h_size, h_size, 3))
    return wave_tensor
    

In [14]:
def wave_tensor_data_backbone(batch_x, depth = -1, i_ref = -1, j_ref = -1):
    batch_size, _ = batch_x.shape
    FLAG_RAND_I = i_ref == -1
    FLAG_RAND_J = j_ref == -1
    FLAG_DEPTH = depth == -1
    wave_tensor = init_wave_tensor(batch_size)
    for num_batch in range(batch_size):
        image = mnist_reshape_32(batch_x[num_batch])
        w = WaveImage(image = image)
        if FLAG_RAND_I:
            i_ref = np.random.randint(16)
        if FLAG_RAND_J:
            j_ref = np.random.randint(16)  
        if FLAG_DEPTH:
            depth = 1 + np.random.randint(6)
        pow2_i = calc_pow2(i_ref)
        pow2_j = calc_pow2(j_ref)
        for h in range(6 - depth, 6):
            data_h = w.get_data()[h]
            if h == 0:
                wave_tensor[h][num_batch][0][0][0] = data_h[(0,0)] #/ 4**4
            else:
                u = (pow2_i[h - 1], pow2_j[h - 1])
                #for u in data_h:
                #    wave_tensor[h][num_batch][u[0]][u[1]][:] = 0
                wave_tensor[h][num_batch][u[0]][u[1]][:] = data_h[u] #/ 4 ** (5 - h)
    return wave_tensor

Construction 
+ 5 couches convolutionnelles : 16 x 16 --> 8 x 8 ; 8 x 8 --> 4 x 4 etc
+ 1 couche FC

In [15]:
DEPTH_WAV = 3
NB_LABEL = 10

In [79]:
mu, Sigma, rho = pickle.load(open("mnist-waveimage-train-mu-Sigma-rho.pkl", "rb"))
#mu, Sigma, rho = pickle.load(open("mnist-waveimage-train-mu-Sigma-rho-naive.pkl", "rb"))
#mu, Sigma, rho = pickle.load(open("mnist-waveimage-train-mu-Sigma-rho-noisy-alt.pkl", "rb"))

In [80]:
# Likelihood calculation (over z's, for given v, h and u)

def calc_lik(v,h,u):
    lik = np.zeros(10)
    for c in range(10):
        if np.linalg.norm(v) < 1e-16:
            if np.linalg.norm(mu[c][h][u]) > 1e-16:
                lik[c] = rho[c][h][u]
            else:
                lik[c] = 1                
        else:
            if np.linalg.norm(mu[c][h][u]) > 1e-16:
                if h == 0:
                    dist = multivariate_normal(mean = mu[c][h][u], cov = Sigma[c][h][u])
                else:
                    dist = multivariate_normal(mean = mu[c][h][u], cov = Sigma[c][h][u] + 1e-10 * np.eye(3))
                lik[c] = (1-rho[c][h][u]) * dist.pdf(v)
                #lik[c] =  dist.pdf(v)
            else:
                lik[c] = 0
        lik[c] = max(lik[c],1e-16)    
    return lik

In [94]:
# Likelihood calculation (over z's, for given v, h and u)

def calc_lik_naive(v,h,u):
    lik = np.zeros(10)
    for c in range(10):
        '''if np.linalg.norm(v) < 1e-16:
            if np.linalg.norm(mu[c][h][u]) > 1e-16:
                lik[c] = rho[c][h][u]
            else:
                lik[c] = 1                
        else:
            if np.linalg.norm(mu[c][h][u]) > 1e-16:'''
        if h == 0:
            dist = multivariate_normal(mean = mu[c][h][u], cov = Sigma[c][h][u])
        else:
            dist = multivariate_normal(mean = mu[c][h][u], cov = np.diag(Sigma[c][h][u] + 1e-10))
            '''lik[c] = (1-rho[c][h][u]) * dist.pdf(v)
            #lik[c] =  dist.pdf(v)
        else:
            lik[c] = 0'''
        lik[c] = max(lik[c], 1e-16)    
    return lik

In [18]:
# Log posterior

def update_log_score(log_score, lik):
    #print 'lik =' + str(lik) 
    log_score += np.log(lik) 
    max_log_score = max(log_score)
    log_score -= max_log_score
    return log_score

In [19]:
# Posterior (Softmax)
    
def calc_pi(log_score): # TODO
    Z = np.sum(np.exp(log_score))
    pi = np.exp(log_score)/Z
    #print 'pi =' + str(pi)
    #print 'max(pi) = ',max(pi)
    return pi

# Parcours predictif

In [21]:
pi_predictive_eff = pickle.load(open("mnist-waveimage-saliency-map.pkl", "rb"))    
#pi_predictive_eff = pickle.load(open("mnist-waveimage-saliency-map-diff-backbone-CNN-parts.pkl", "rb"))    

In [22]:
H_generic_eff = pickle.load(open("mnist-waveimage-generic-saliency-map.pkl", "rb"))    
#pi_predictive_eff = pickle.load(open("mnist-waveimage-saliency-map-diff-backbone-CNN-parts.pkl", "rb"))    

In [23]:
def calcule_asc_path(h,u):
    rep = []
    for h_inf in range(h, 0, -1):
        i_inf = u[0] / (2 ** (h - h_inf))
        j_inf = u[1] / (2 ** (h - h_inf))
        rep += [(h_inf, (i_inf, j_inf))]
    # racine
    rep += [(0, (i_inf, j_inf))]
    return rep

In [24]:
def calc_pi_predictive_sorted(pi_predictive): 
    pi_predictive_sorted = {}
    for c in range(10):
        pi_predictive_sorted[c] = []
        for k in pi_predictive[c]:
            pi_predictive_sorted[c] += [(pi_predictive[c][k], k)]
        pi_predictive_sorted[c] = sorted(pi_predictive_sorted[c])
    return pi_predictive_sorted

In [25]:
def calc_H_predictive_sorted(H_predictive): 
    H_predictive_sorted = []
    for k in H_predictive:
        H_predictive_sorted += [(H_predictive[k], k)]
    H_predictive_sorted = sorted(H_predictive_sorted, reverse=True)
    return H_predictive_sorted

In [26]:
def argmax_generator(c, h, u):
    test_pred = rho[c][h][u] < .5       
    if test_pred:
        return mu[c][h][u]
    else:
        return np.zeros(3)

In [27]:
def softmax_generator(log_score, h, u):
    Z = np.sum(np.exp(log_score))
    mu_c = np.zeros(3)
    for c in range(NB_LABEL):
        pi = np.exp(log_score[0][c]) / Z
        mu_c += pi * mu[c][h][u] * (1 - rho[c][h][u])
    return mu_c

In [28]:
axes = []
h_max = 6
shape = (32,32)

U = {}
for h in range(h_max):
    U [h] = {}
    dim_i, dim_j = calc_dim(shape, h, h_max)
    for i in range(dim_i):
        for j in range(dim_j):
            U[h][(i,j)] = 1    

In [98]:
h_max = 6
lik_predictive = {}
for c in range(10):
    lik_predictive[c] = {}
    for h in range(h_max):
        lik_predictive[c][h] = {}
        for u in U[h]:
            v_predictive = argmax_generator(c, h, u)
            #lik = calc_lik(v_predictive, h, u)
            lik = calc_lik_naive(v_predictive, h, u)
            lik_predictive[c][h][u] = lik

In [30]:
def predictive_search(z_ref, log_score, actions_set, mem_h_u, FLAG_DUAL = False, FLAG_KL = False):
    # actions_set ne contient que les positions de niveau 5
    h_ref = 5
    batch_size = len(actions_set)
    #print batch_size
    ## Parcours predictif
    log_score_path = np.zeros((batch_size, NB_LABEL))
    for i, u_gen in enumerate(actions_set):
        log_score_path[i] = np.copy(log_score[0])
    dict_u = {}
    for i, u_gen in enumerate(actions_set):
        dict_u[i] = u_gen
        liste_path = calcule_asc_path(h_ref, u_gen)
        #print(liste_path)
        for (h_path, u_path) in liste_path[:-1]:
            #print(h_path, u_path)
            if (h_path, u_path) not in mem_h_u:
                log_score_path[i] = update_log_score(log_score_path[i], lik_predictive[z_ref][h_path][u_path])
        #print log_score_path[i]
        
    q_pre = np.exp(log_score[0]) / np.sum(np.exp(log_score[0]))
    FEP_post = np.zeros(batch_size)
    for i, u_gen in enumerate(actions_set):
        q_post = np.exp(log_score_path[i]) / np.sum(np.exp(log_score_path[i]))  
        if not FLAG_DUAL:
            FEP_post[i] = entropy(q_post)
        else:
            delta_log_score_path = log_score_path[i] - log_score[0]
            delta_q_post = np.exp(delta_log_score_path) / np.sum(np.exp(delta_log_score_path))
            if not FLAG_KL:
                FEP_post[i] = entropy(q_post) - np.log(delta_q_post[z_ref]) + np.log(q_pre[z_ref]) 
            else:
                FEP_post[i] = - np.log(delta_q_post[z_ref]) + np.log(q_pre[z_ref]) 
            #FEP_post[i] = - np.log(delta_q_post[z_ref]) #+ np.log(q_pre[c])
            #print u_gen, entropy(q_post), - np.log(delta_q_post[z_ref]), FEP_post[i] 
                
    #i_max = np.where(log_score_path[:, z_ref] == max(log_score_path[:, z_ref]))[0][0]
    i_min = np.where(FEP_post == min(FEP_post))[0][0]
    q_post = np.exp(log_score_path[i_min]) / np.sum(np.exp(log_score_path[i_min])) 
    delta_log_score_path = log_score_path[i_min] - log_score[0]
    delta_q_post = np.exp(delta_log_score_path) / np.sum(np.exp(delta_log_score_path))
    #print log_score_path[i_min] 
    #print dict_u[i_min] , entropy(q_post), - np.log(delta_q_post[z_ref]) + np.log(q_pre[z_ref]), FEP_post[i_min]
    
    
    ## 3 ##
    return dict_u[i_min] #, pi_path[i_max] #pi_path[i_max][z_ref]
    

In [82]:
def FEP_predictive_search_full(log_score, actions_set, mem_h_u, FLAG_DUAL = False, FLAG_KL = False):
    # actions_set ne contient que les positions de niveau 5
    h_ref = 5
    batch_ref = len(actions_set)
    batch_size = batch_ref * NB_LABEL
    log_score_path = {}
    
    ## Parcours predictif
    log_score_path = np.zeros((batch_size, NB_LABEL))
    for i, u_gen in enumerate(actions_set):
        for c in range(NB_LABEL):
            i_full = c * batch_ref + i
            log_score_path[i_full] = np.copy(log_score[0])
    dict_u = {}
    for i, u_gen in enumerate(actions_set):
        dict_u[i] = u_gen
        liste_path = calcule_asc_path(h_ref, u_gen)
        #print(liste_path)
        for (h_path, u_path) in liste_path[:-1]:
            #print(h_path, u_path)
            if (h_path, u_path) not in mem_h_u:
                for c in range(NB_LABEL):
                    i_full = c * batch_ref + i
                    log_score_path[i_full] = update_log_score(log_score_path[i_full],\
                                                              lik_predictive[c][h_path][u_path])
                        
    FEP_post = np.zeros(batch_size)
    q_pre = np.exp(log_score[0]) / np.sum(np.exp(log_score[0]))
    '''log_score_post_full = np.zeros((batch_ref,))
    for c in range(NB_LABEL):
        log_score_post_full += q_pre[c] * log_score_path[i_full]    #print q_pre'''
        
    for i, u_gen in enumerate(actions_set):
        for c in range(NB_LABEL):
            i_full = c * batch_ref + i
            q_post = np.exp(log_score_path[i_full]) / np.sum(np.exp(log_score_path[i_full]))  
            if not FLAG_DUAL:
                FEP_post[i_full] = entropy(q_post)
                #print u_gen, q_post
                #print FEP_post[i]
            else:
                delta_log_score_path = log_score_path[i_full] - log_score[0]
                delta_q_post = np.exp(delta_log_score_path) / np.sum(np.exp(delta_log_score_path))
                #q_pre_hat = np.zeros(NB_LABEL)
                #q_pre_hat[c] = 1
                if not FLAG_KL:
                    if q_pre[c] > 1e-16:
                        FEP_post[i_full] = entropy(q_post) - np.log(delta_q_post[c]) + np.log(q_pre[c])
                    else:
                        FEP_post[i_full] = entropy(q_post) - np.log(delta_q_post[c]) + np.log(1e-16)
                else:
                    FEP_post[i_full] = - np.log(delta_q_post[c]) + np.log(q_pre[c])
                if not np.isfinite(FEP_post[i_full]):
                    print "aie!!"
                    FEP_post[i_full] = 50
                #FEP_post[i_full] = - np.log(delta_q_post[c]) #+ np.log(q_pre[c])
                #FEP_post[i_full] = entropy(q_post) + entropy(q_pre, delta_q_post)
                #print u_gen, entropy(q_post), - np.log(delta_q_post[c]), FEP_post[i_full] 
    #ch = raw_input('')    
    
    FEP_post_full = np.zeros((batch_ref,))
    for c in range(NB_LABEL):
        FEP_post_full += q_pre[c] * FEP_post[c * batch_ref : (c + 1) * batch_ref]
        
    #for i, u_gen in enumerate(actions_set):
    #    print u_gen, FEP_post_full[i]
    
    i_min = np.where(FEP_post_full == min(FEP_post_full))[0][0]
    
    ## 3 ##
    return dict_u[i_min]  #pi_path[i_max][z_ref]
    

In [32]:
def prediction_based_policy(log_score, actions_set, mem_h_u, FLAG_DUAL = False, FLAG_KL = False):
    
    ## 1 ##
    z_tilde = np.argmax(log_score)    
    u_tilde = predictive_search(z_tilde, log_score, actions_set, mem_h_u, FLAG_DUAL, FLAG_KL)
    
    return u_tilde

In [33]:
def FEP_prediction_based_policy(log_score, actions_set, mem_h_u, FLAG_DUAL = False, FLAG_FULL = False, FLAG_KL = False):
    
    ## 1 ##
    if not FLAG_FULL:
        u_tilde = FEP_predictive_search(log_score, actions_set, mem_h_u, FLAG_DUAL, FLAG_KL)
    else:
        u_tilde = FEP_predictive_search_full(log_score, actions_set, mem_h_u, FLAG_DUAL, FLAG_KL)
    
    return u_tilde

In [34]:
def saliency_based_policy(log_score, pi_predictive_sorted, mem_h_u):
    h = 5
    ## 1 ##
    z_tilde = np.argmax(log_score)
    u_tilde = pi_predictive_sorted[z_tilde].pop()[1]
    while (h, u_tilde) in mem_h_u:
        u_tilde = pi_predictive_sorted[z_tilde].pop()[1]
    return u_tilde
    

In [35]:
def generic_saliency_based_policy(H_predictive_sorted, mem_h_u):
    u_tilde = H_predictive_sorted.pop()[1]
    return u_tilde

In [36]:
def random_policy(log_score, mem_h_u):
    h = 5
    ## 1 ##
    z_tilde = np.argmax(log_score)
    #u_tilde = (1 + np.random.randint(14),  1 + np.random.randint(14))
    u_tilde = (np.random.randint(16),  np.random.randint(16))
    while (h, u_tilde) in mem_h_u:
        #u_tilde = (1 + np.random.randint(14),  1 + np.random.randint(14))
        u_tilde = (np.random.randint(16),  np.random.randint(16))
    return u_tilde

In [37]:
def scene_exploration(wave_tensor_ref, log_score, z_ref, ind_test, actions_set, mem_h_u, record, \
                      POL = 'predictive', AFF = True, THRESHOLD = 1e-4):
    
    assert POL == 'predictive' or POL == 'predictive-dual' or POL == 'predictive-KL'\
    or POL == 'saliency-based' or POL == 'random' or POL == 'full' or POL == 'generic-saliency-based'\
    or POL == 'FEP-predictive' or POL == 'FEP-predictive-dual' or POL == 'FEP-predictive-KL'
    
    if POL == 'full':
        THRESHOLD = 0
        POL = 'saliency-based'
    
    TOUR = 0
    END = False
    h_ref = 5
    
    # saliency-based approach
    if POL == 'saliency-based':
        pi_predictive_sorted = calc_pi_predictive_sorted(pi_predictive_eff)
    if True : #POL == 'generic-saliency-based':
        H_predictive_sorted = calc_H_predictive_sorted(H_generic_eff)    
    while END == False:
        
        if AFF:
            print '************************************'
            print '******       TOUR    ' + str(TOUR + 1) + '        ******'
            print '************************************'
        
        # 1. CHOIX
        if POL == 'predictive':
            if TOUR == 0:
                u_tilde = H_predictive_sorted[-1][1]
            else:
                u_tilde = prediction_based_policy(log_score, actions_set, mem_h_u)
        elif POL == 'predictive-dual':
            if TOUR == 0:
                u_tilde = H_predictive_sorted[-1][1]
            else:
                u_tilde = prediction_based_policy(log_score, actions_set, mem_h_u, FLAG_DUAL = True)
        elif POL == 'predictive-KL':
            if TOUR == 0:
                u_tilde = H_predictive_sorted[-1][1]
            else:
                u_tilde = prediction_based_policy(log_score, actions_set, mem_h_u, FLAG_DUAL = True, FLAG_KL = True)
            #print u_tilde
        elif POL == 'FEP-predictive-dual':
            if TOUR == 0:
                u_tilde = H_predictive_sorted[-1][1]
            else:
                u_tilde = FEP_prediction_based_policy(log_score, actions_set, mem_h_u,\
                                                      FLAG_DUAL = True, FLAG_FULL = True)
            #print u_tilde
            #ch = raw_input('')     
        elif POL == 'FEP-predictive':
            if TOUR == 0:
                u_tilde = H_predictive_sorted[-1][1]
            else:
                u_tilde = FEP_prediction_based_policy(log_score, actions_set, mem_h_u,\
                                                      FLAG_DUAL = False, FLAG_FULL = True)
            #print u_tilde, H_predictive_sorted[-1]
        elif POL == 'FEP-predictive-KL':
            if TOUR == 0:
                u_tilde = H_predictive_sorted[-1][1]
            else:
                u_tilde = FEP_prediction_based_policy(log_score, actions_set, mem_h_u,\
                                                      FLAG_DUAL = True, FLAG_FULL = True, FLAG_KL = True)    
            #print u_tilde
            #ch = raw_input('')    
        elif POL == 'saliency-based':
            if TOUR == 0:
                u_tilde = H_predictive_sorted[-1][1]
            else:
                u_tilde = saliency_based_policy(log_score, pi_predictive_sorted, mem_h_u)
        elif POL == 'generic-saliency-based':
            u_tilde = generic_saliency_based_policy(H_predictive_sorted, mem_h_u)
        else:
            u_tilde = random_policy(log_score, mem_h_u)
        
        if AFF:
            print 'CHOIX :', u_tilde
        
        # 2. LECTURE + UPDATE
        #wave_tensor =  init_wave_tensor(1)
        liste_path = calcule_asc_path(h_ref, u_tilde)
        
        for (h_path, u_path) in reversed(liste_path):
            if (h_path, u_path) not in mem_h_u:
                v = wave_tensor_ref[h_path][0][u_path[0]][u_path[1]][:]
                lik = calc_lik(v, h_path, u_path)
                log_score[0] = update_log_score(log_score[0], lik)
                #wave_tensor[h_path][0][u_path[0]][u_path[1]][:] =  
                mem_h_u += [(h_path, u_path)]
                record.mem_h_u += [(h_path, u_path)]
                record.nb_coeffs += 3  
                
        pi = np.exp(log_score[0]) / np.sum(np.exp(log_score[0])) #sess.run(tf.nn.softmax(log_score))[0]
                
        H = entropy(pi) # sess.run(tf.nn.softmax_cross_entropy_with_logits(labels=pi,logits=log_score)) #np.sum(- pi * np.log(pi))
        out = np.argmax(pi)
                
        if AFF :
            print 'pi : ', pi
            print 'out :', out
            print 'pi[out] : ', pi[out]
            print 'H : ', H

        record.mem_pi += [pi]
        record.mem_H += [H]
        record.mem_z += [out]
                
        # 3. INHIBITION OF RETURN        
        actions_set.pop(u_tilde)
        
        record.mem_u += [u_tilde]
        record.nb_saccades += 1
                
        if AFF:
            print '****', 'z :', z_ref, ', u :',u_tilde, ' ---> ', out
              
        
        if TOUR == 16 * 16 - 1 or H < THRESHOLD:
            END = True
            if AFF :
                print '************************************'
                print '******         FINI          *******'
                print '************************************' 
            return out
        else:
            TOUR += 1   

## Main

In [38]:
from record import Record, affiche_records            

In [39]:
NB_TRIALS = 1000

In [40]:
mnist = input_data.read_data_sets("MNIST_data/")

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


    Policy : predictive, threshold : 0.03, 4 saccades, initial : 3, final : 7, classe : 7, elapsed time : 4.96038
    Policy : predictive, threshold : 0.03, 4 saccades, initial : 0, final : 2, classe : 2, elapsed time : 9.72888
    Policy : predictive, threshold : 0.03, 1 saccades, initial : 1, final : 1, classe : 1, elapsed time : 11.6165
    Policy : predictive, threshold : 0.03, 2 saccades, initial : 0, final : 0, classe : 0, elapsed time : 14.5421
    Policy : predictive, threshold : 0.03, 8 saccades, initial : 4, final : 4, classe : 4, elapsed time : 22.9571
    Policy : predictive, threshold : 0.03, 1 saccades, initial : 1, final : 1, classe : 1, elapsed time : 24.8637
    Policy : predictive, threshold : 0.03, 3 saccades, initial : 7, final : 4, classe : 4, elapsed time : 28.7859
    Policy : predictive, threshold : 0.03, 4 saccades, initial : 8, final : 9, classe : 9, elapsed time : 33.8153
    Policy : predictive, threshold : 0.03, 10 saccades, initial : 2, final : 5, classe : 5, elapsed time : 44.5951
    Policy : predictive, threshold : 0.03, 3 saccades, initial : 0, final : 9, classe : 9, elapsed time : 48.4158

In [72]:
# Test generic saliency map

In [ ]:
import time
dict_records = {}

#file_name = "mnist-waveimage-CNN-backbone-records-rnd-parts-generic-saliency.npy"
file_name = "mnist-waveimage-records-FEP-dual-full-H0_init-naive-bayes.npy"
#file_name = "tmp"

if True : #not os.path.isfile(file_name):
    
    for POL in ('FEP-predictive-dual', 'FEP-predictive'):#, 'FEP-predictive-KL', 'saliency-based', \
               # 'generic-saliency-based', 'random', 'predictive-KL', 'predictive-dual', 'predictive',): # 'FEP-predictive'): #'generic-saliency-based',): 

        dict_records[POL] = {}

        for THRESHOLD in (1e-1, 1e-2, 1e-3, 1e-4, 1e-5):
        #for THRESHOLD in (2, 1.5, 1, 7e-1, 5e-1, 3e-1, 2e-1, 1e-1):# 
        #for THRESHOLD in (1, 3e-1, 1e-1, 3e-2, 1e-2): #, 1e-5):

            records = [] 
            cpt_TRIALS = 0

            tic = time.time()

            #NB_TRIALS = len(Data_test[z_ref][0][(0,0)])

            for ind_test in range(NB_TRIALS):
                if ind_test % 10 == 0:
                    print(POL, THRESHOLD, ind_test)
                x_test, z_ref = mnist.test.images[ind_test], mnist.test.labels[ind_test]
                #wave_tensor = wave_tensor_data(x_test)
                #m_test = np.sum(x_test) 
                #x_noise =  x_test * .9  + .1 * ((1 - x_test) * np.random.rand(28 * 28))
                #x_noise = x_test * .9  + .1 * ((1 - x_test))
                #x_dim = x_test * .5
                #x_noise = x_noise * np.random.rand(28 * 28) * np.random.binomial(1, x_noise) #+ np.random.binomial(1, x_noise) * np.random.rand(28 * 28) - .5
                #x_noise = x_noise * np.random.binomial(1, x_noise) #+ np.random.binomial(1, x_noise) * np.random.rand(28 * 28) - .5
                #m_noise = np.sum(x_noise)
                # x_noise = x_noise * m_test / m_noise
                x_noise = np.copy(x_test)
                '''for _ in range(75):
                    i_1 = np.where(np.random.multinomial(1, np.ones(784)/784) == 1)[0][0]
                    i_2 = np.where(np.random.multinomial(1, np.ones(784)/784) == 1)[0][0]
                    x_noise[i_1], x_noise[i_2] = x_noise[i_2], x_noise[i_1]'''
                '''m_x = np.mean(x_test)
                x_noise = x_test * (1 - .4)  + .4 * (1 - x_test)
                x_noise = x_noise * np.random.rand(28 * 28) * np.random.binomial(1, x_noise)'''# + 0.1 * np.random.randn(28 * 28)
                wave_tensor_ref = wave_tensor_data(np.reshape(x_noise , (1, 28*28)))
                          # 
                # initial
                log_score = np.zeros((1,10))
                pi = np.ones(10) / 10
                H = entropy(pi) #np.sum(- pi * np.log(pi))

                record = Record()
                record.POL = POL
                record.THRESHOLD = THRESHOLD
                record.z_ref = z_ref
                record.mem_pi += [pi]
                record.mem_H += [H]
                z_tilde = -1

                # global coef --> log_score initial
                '''mem_h_u = [(0, (0, 0)), (1, (0, 0))]
                h, u = 0, (0, 0)
                v = wave_tensor_ref[h][0][u[0]][u[1]][:]
                lik = calc_lik(v,h,u)
                log_score = update_log_score(log_score, lik)
                h, u = 1, (0, 0)
                v = wave_tensor_ref[h][0][u[0]][u[1]][:]
                lik = calc_lik(v,h,u)
                log_score = update_log_score(log_score, lik)
                pi = calc_pi(log_score)
                H = entropy(pi) #np.sum(- pi * np.log(pi))
                z_tilde = np.argmax(log_score)

                record.mem_pi += [pi]
                record.mem_H += [H]
                record.mem_z += [z_tilde]
                record.nb_coeffs += 1'''
                mem_h_u = []

                # initial actions set
                actions_set = {}
                for i in range(16):
                    for j in range(16):
                        actions_set[(i, j)] = 1
                
                if ind_test == 220:
                    AFF = True
                else:
                    AFF = False

                z_final = scene_exploration(wave_tensor_ref, log_score, z_ref, ind_test,\
                                            actions_set, mem_h_u, record, \
                                            POL = POL, AFF = AFF, THRESHOLD = THRESHOLD)
                record.z_final = z_final
                record.success = z_ref == z_final

                records += [record]

                toc = time.time()
                if NB_TRIALS <= 1000:
                    print '\rPolicy : %s, threshold : %g, %d saccades, initial : %d, final : %d, classe : %d, elapsed time : %g' \
                                    % (POL, THRESHOLD, record.nb_saccades, z_tilde, z_final, z_ref, toc - tic)   
                cpt_TRIALS  += NB_TRIALS

            dict_records[POL][THRESHOLD] = records
            print '\n'
            print 'Nb trials :', cpt_TRIALS
            #affiche_records(records)
            print '\n'
            np.save(file_name, dict_records)
else:
    dict_records = np.load(file_name)
        

('FEP-predictive-dual', 0.1, 0)
Policy : FEP-predictive-dual, threshold : 0.1, 4 saccades, initial : -1, final : 7, classe : 7, elapsed time : 0.476384
Policy : FEP-predictive-dual, threshold : 0.1, 1 saccades, initial : -1, final : 6, classe : 2, elapsed time : 0.494418
Policy : FEP-predictive-dual, threshold : 0.1, 1 saccades, initial : -1, final : 1, classe : 1, elapsed time : 0.512777
Policy : FEP-predictive-dual, threshold : 0.1, 1 saccades, initial : -1, final : 0, classe : 0, elapsed time : 0.529044
Policy : FEP-predictive-dual, threshold : 0.1, 2 saccades, initial : -1, final : 4, classe : 4, elapsed time : 0.689966
Policy : FEP-predictive-dual, threshold : 0.1, 1 saccades, initial : -1, final : 1, classe : 1, elapsed time : 0.707054
Policy : FEP-predictive-dual, threshold : 0.1, 15 saccades, initial : -1, final : 9, classe : 4, elapsed time : 2.63809
Policy : FEP-predictive-dual, threshold : 0.1, 4 saccades, initial : -1, final : 9, classe : 9, elapsed time : 3.16775
Policy : 

Policy : FEP-predictive-dual, threshold : 0.1, 22 saccades, initial : -1, final : 3, classe : 3, elapsed time : 37.6655
Policy : FEP-predictive-dual, threshold : 0.1, 2 saccades, initial : -1, final : 0, classe : 0, elapsed time : 37.8242
('FEP-predictive-dual', 0.1, 70)
Policy : FEP-predictive-dual, threshold : 0.1, 1 saccades, initial : -1, final : 7, classe : 7, elapsed time : 37.8395
Policy : FEP-predictive-dual, threshold : 0.1, 1 saccades, initial : -1, final : 0, classe : 0, elapsed time : 37.8525
Policy : FEP-predictive-dual, threshold : 0.1, 2 saccades, initial : -1, final : 2, classe : 2, elapsed time : 38.0118
Policy : FEP-predictive-dual, threshold : 0.1, 17 saccades, initial : -1, final : 7, classe : 9, elapsed time : 40.1201
Policy : FEP-predictive-dual, threshold : 0.1, 1 saccades, initial : -1, final : 1, classe : 1, elapsed time : 40.1392
Policy : FEP-predictive-dual, threshold : 0.1, 3 saccades, initial : -1, final : 4, classe : 7, elapsed time : 40.4813
Policy : FEP-

Policy : FEP-predictive-dual, threshold : 0.1, 5 saccades, initial : -1, final : 0, classe : 0, elapsed time : 79.8027
Policy : FEP-predictive-dual, threshold : 0.1, 1 saccades, initial : -1, final : 1, classe : 1, elapsed time : 79.8208
Policy : FEP-predictive-dual, threshold : 0.1, 1 saccades, initial : -1, final : 6, classe : 6, elapsed time : 79.8386
Policy : FEP-predictive-dual, threshold : 0.1, 1 saccades, initial : -1, final : 4, classe : 4, elapsed time : 79.8547
('FEP-predictive-dual', 0.1, 140)
Policy : FEP-predictive-dual, threshold : 0.1, 4 saccades, initial : -1, final : 6, classe : 6, elapsed time : 80.2955
Policy : FEP-predictive-dual, threshold : 0.1, 1 saccades, initial : -1, final : 7, classe : 7, elapsed time : 80.3115
Policy : FEP-predictive-dual, threshold : 0.1, 22 saccades, initial : -1, final : 3, classe : 3, elapsed time : 82.9984
Policy : FEP-predictive-dual, threshold : 0.1, 1 saccades, initial : -1, final : 1, classe : 1, elapsed time : 83.0165
Policy : FEP-

Policy : FEP-predictive-dual, threshold : 0.1, 1 saccades, initial : -1, final : 1, classe : 1, elapsed time : 122.35
Policy : FEP-predictive-dual, threshold : 0.1, 2 saccades, initial : -1, final : 3, classe : 3, elapsed time : 122.516
Policy : FEP-predictive-dual, threshold : 0.1, 2 saccades, initial : -1, final : 9, classe : 9, elapsed time : 122.687
Policy : FEP-predictive-dual, threshold : 0.1, 25 saccades, initial : -1, final : 5, classe : 5, elapsed time : 125.792
Policy : FEP-predictive-dual, threshold : 0.1, 1 saccades, initial : -1, final : 2, classe : 2, elapsed time : 125.808
Policy : FEP-predictive-dual, threshold : 0.1, 5 saccades, initial : -1, final : 9, classe : 9, elapsed time : 126.378
('FEP-predictive-dual', 0.1, 210)
Policy : FEP-predictive-dual, threshold : 0.1, 2 saccades, initial : -1, final : 4, classe : 4, elapsed time : 126.537
Policy : FEP-predictive-dual, threshold : 0.1, 14 saccades, initial : -1, final : 7, classe : 5, elapsed time : 128.376
Policy : FEP-

Policy : FEP-predictive-dual, threshold : 0.1, 17 saccades, initial : -1, final : 7, classe : 7, elapsed time : 159.594
Policy : FEP-predictive-dual, threshold : 0.1, 17 saccades, initial : -1, final : 7, classe : 7, elapsed time : 162.089
Policy : FEP-predictive-dual, threshold : 0.1, 4 saccades, initial : -1, final : 1, classe : 2, elapsed time : 162.613
Policy : FEP-predictive-dual, threshold : 0.1, 1 saccades, initial : -1, final : 1, classe : 8, elapsed time : 162.633
Policy : FEP-predictive-dual, threshold : 0.1, 1 saccades, initial : -1, final : 2, classe : 2, elapsed time : 162.649
Policy : FEP-predictive-dual, threshold : 0.1, 5 saccades, initial : -1, final : 0, classe : 6, elapsed time : 163.387
('FEP-predictive-dual', 0.1, 260)
Policy : FEP-predictive-dual, threshold : 0.1, 2 saccades, initial : -1, final : 8, classe : 8, elapsed time : 163.594
Policy : FEP-predictive-dual, threshold : 0.1, 18 saccades, initial : -1, final : 5, classe : 5, elapsed time : 166.182
Policy : FE

...

CHOIX : (10, 5)
pi :  [  1.60996896e-081   1.44400564e-209   4.46169201e-102   4.66481066e-054
   1.52241944e-026   3.03799761e-048   4.21465289e-128   5.52227092e-003
   3.37472203e-054   9.94477729e-001]
out : 9
pi[out] :  0.99447772908
H :  0.0342170943688
**** z : 7 , u : (10, 5)  --->  9
************************************
******       TOUR    180        ******
************************************
CHOIX : (7, 0)
pi :  [  1.60996896e-081   1.44400564e-209   4.46169201e-102   4.66481066e-054
   1.52241944e-026   3.03799761e-048   4.21465289e-128   5.52227092e-003
   3.37472203e-054   9.94477729e-001]
out : 9
pi[out] :  0.99447772908
H :  0.0342170943688
**** z : 7 , u : (7, 0)  --->  9
************************************
******       TOUR    181        ******
************************************
CHOIX : (6, 8)
pi :  [  2.00110689e-081   2.20519630e-210   2.50369204e-102   3.26705370e-054
   1.13063683e-026   3.31359687e-048   2.41577370e-128   1.02423711e-002
   1.97497933e-054   

CHOIX : (15, 8)
pi :  [  8.02397930e-090   1.35362695e-226   1.59610881e-108   2.23441469e-059
   1.18727468e-028   7.52558290e-052   8.82850083e-140   5.31925855e-004
   3.52512996e-058   9.99468074e-001]
out : 9
pi[out] :  0.999468074145
H :  0.00454197680772
**** z : 7 , u : (15, 8)  --->  9
************************************
******       TOUR    207        ******
************************************
CHOIX : (14, 0)
pi :  [  8.02397930e-090   1.35362695e-226   1.59610881e-108   2.23441469e-059
   1.18727468e-028   7.52558290e-052   8.82850083e-140   5.31925855e-004
   3.52512996e-058   9.99468074e-001]
out : 9
pi[out] :  0.999468074145
H :  0.00454197680772
**** z : 7 , u : (14, 0)  --->  9
************************************
******       TOUR    208        ******
************************************
CHOIX : (13, 8)
pi :  [  1.43835367e-089   2.50137658e-226   3.40175833e-108   3.04586925e-059
   1.64454931e-028   1.06478668e-051   1.93839277e-139   5.67635706e-004
   3.64670350e

**** z : 7 , u : (6, 12)  --->  9
************************************
******       TOUR    239        ******
************************************
CHOIX : (11, 12)
pi :  [  2.00906000e-094   5.76739508e-231   4.36768359e-111   1.82100927e-060
   2.74384958e-030   5.51899463e-054   7.26905665e-144   1.61224368e-005
   4.06358106e-060   9.99983878e-001]
out : 9
pi[out] :  0.999983877563
H :  0.000194038212666
**** z : 7 , u : (11, 12)  --->  9
************************************
******       TOUR    240        ******
************************************
CHOIX : (7, 9)
pi :  [  2.57945410e-094   3.37184347e-234   5.87086247e-111   2.48377090e-062
   9.03490360e-030   6.39807193e-058   2.64491681e-146   6.62732694e-005
   3.90465173e-062   9.99933727e-001]
out : 9
pi[out] :  0.999933726731
H :  0.000703934174996
**** z : 7 , u : (7, 9)  --->  9
************************************
******       TOUR    241        ******
************************************
CHOIX : (14, 11)
pi :  [  2.65189

Policy : FEP-predictive-dual, threshold : 1e-05, 25 saccades, initial : -1, final : 8, classe : 8, elapsed time : 955.677
Policy : FEP-predictive-dual, threshold : 1e-05, 28 saccades, initial : -1, final : 4, classe : 4, elapsed time : 959.085
Policy : FEP-predictive-dual, threshold : 1e-05, 2 saccades, initial : -1, final : 1, classe : 1, elapsed time : 959.272
Policy : FEP-predictive-dual, threshold : 1e-05, 27 saccades, initial : -1, final : 7, classe : 7, elapsed time : 962.501
('FEP-predictive-dual', 1e-05, 230)
Policy : FEP-predictive-dual, threshold : 1e-05, 13 saccades, initial : -1, final : 3, classe : 3, elapsed time : 964.229
Policy : FEP-predictive-dual, threshold : 1e-05, 24 saccades, initial : -1, final : 3, classe : 3, elapsed time : 967.297
Policy : FEP-predictive-dual, threshold : 1e-05, 25 saccades, initial : -1, final : 8, classe : 8, elapsed time : 970.538
Policy : FEP-predictive-dual, threshold : 1e-05, 183 saccades, initial : -1, final : 7, classe : 8, elapsed tim

Policy : FEP-predictive-dual, threshold : 1e-05, 18 saccades, initial : -1, final : 9, classe : 9, elapsed time : 1241.33
Policy : FEP-predictive-dual, threshold : 1e-05, 2 saccades, initial : -1, final : 2, classe : 2, elapsed time : 1241.62
Policy : FEP-predictive-dual, threshold : 1e-05, 27 saccades, initial : -1, final : 0, classe : 0, elapsed time : 1245.22
Policy : FEP-predictive-dual, threshold : 1e-05, 52 saccades, initial : -1, final : 4, classe : 4, elapsed time : 1251.92
Policy : FEP-predictive-dual, threshold : 1e-05, 2 saccades, initial : -1, final : 0, classe : 0, elapsed time : 1252.09
Policy : FEP-predictive-dual, threshold : 1e-05, 5 saccades, initial : -1, final : 0, classe : 0, elapsed time : 1252.85
Policy : FEP-predictive-dual, threshold : 1e-05, 24 saccades, initial : -1, final : 2, classe : 2, elapsed time : 1256.25
Policy : FEP-predictive-dual, threshold : 1e-05, 24 saccades, initial : -1, final : 8, classe : 8, elapsed time : 1259.8
('FEP-predictive-dual', 1e-0

Policy : FEP-predictive, threshold : 0.1, 2 saccades, initial : -1, final : 7, classe : 7, elapsed time : 0.290342
Policy : FEP-predictive, threshold : 0.1, 1 saccades, initial : -1, final : 6, classe : 2, elapsed time : 0.443286
Policy : FEP-predictive, threshold : 0.1, 1 saccades, initial : -1, final : 1, classe : 1, elapsed time : 0.587564
Policy : FEP-predictive, threshold : 0.1, 1 saccades, initial : -1, final : 0, classe : 0, elapsed time : 0.736176
Policy : FEP-predictive, threshold : 0.1, 2 saccades, initial : -1, final : 4, classe : 4, elapsed time : 0.991568
Policy : FEP-predictive, threshold : 0.1, 1 saccades, initial : -1, final : 1, classe : 1, elapsed time : 1.14426
Policy : FEP-predictive, threshold : 0.1, 2 saccades, initial : -1, final : 4, classe : 4, elapsed time : 1.446
Policy : FEP-predictive, threshold : 0.1, 2 saccades, initial : -1, final : 9, classe : 9, elapsed time : 1.69376
Policy : FEP-predictive, threshold : 0.1, 2 saccades, initial : -1, final : 6, classe : 5, elapsed time : 1.95663
Policy : FEP-predictive, threshold : 0.1, 3 saccades, initial : -1, final : 9, classe : 9, elapsed time : 2.32355

## Main Backup

In [166]:
import time
dict_records = {}

file_name = mnist-waveimage-CNN-backbone-records-rnd-parts-FEP.npy"

if not os.path.isfile(file_name):
    
    for POL in ('FEP-predictive',): # 'saliency-based',  'random'): 

        dict_records[POL] = {}

        for THRESHOLD in (1, 3e-1, 1e-1, 3e-2, 1e-2):

            records = [] 
            cpt_TRIALS = 0

            tic = time.time()

            #NB_TRIALS = len(Data_test[z_ref][0][(0,0)])

            for ind_test in range(NB_TRIALS):
                if ind_test % 10 == 0:
                    print(POL, THRESHOLD, ind_test)
                x_test, z_ref = mnist.test.images[ind_test], mnist.test.labels[ind_test]
                #wave_tensor = wave_tensor_data(x_test)
                wave_tensor_ref = wave_tensor_data(np.reshape(x_test,(1, 28*28)))

                # initial
                log_score = np.zeros(10)
                pi = np.ones(10) / 10
                H = entropy(pi) #np.sum(- pi * np.log(pi))

                record = Record()
                record.POL = POL
                record.THRESHOLD = THRESHOLD
                record.z_ref = z_ref
                record.mem_pi += [pi]
                record.mem_H += [H]

                # global coef --> log_score initial
                h, u = 0, (0, 0)
                mem_h_u = [(0, (0, 0)), (1, (0, 0))]
                wave_tensor =  init_wave_tensor(1)
                wave_tensor[0][0][0][0][0] =  wave_tensor_ref[0][0][0][0][0]
                wave_tensor[1][0][0][0][:] =  wave_tensor_ref[1][0][0][0][:]
                log_score = y_hat_logit.eval(feed_dict={x_5: wave_tensor[5],\
                            x_4: wave_tensor[4],\
                            x_3: wave_tensor[3],\
                            x_2: wave_tensor[2],\
                            x_1: wave_tensor[1],\
                            x_0: wave_tensor[0],\
                            keep_prob: 1,\
                            batch_phase:False})  
                pi = sess.run(tf.nn.softmax(log_score))[0]
                H = entropy(pi) #H = sess.run(tf.nn.softmax_cross_entropy_with_logits(labels=pi,logits=log_score)) #np.sum(- pi * np.log(pi))
                z_tilde = np.argmax(pi)

                record.mem_pi += [pi]
                record.mem_H += [H]
                record.mem_z += [z_tilde]
                record.nb_coeffs += 1

                # initial actions set
                actions_set = {}
                for i in range(16):
                    for j in range(16):
                        actions_set[(i, j)] = 1

                z_final = scene_exploration(sess, wave_tensor_ref, wave_tensor, log_score, z_ref, ind_test,\
                                            actions_set, mem_h_u, record, \
                                            POL = POL, AFF = False, THRESHOLD = THRESHOLD)
                record.z_final = z_final
                record.success = z_ref == z_final

                records += [record]

                toc = time.time()
                print '\rPolicy : %s, threshold : %g, %d saccades, initial : %d, final : %d, classe : %d, elapsed time : %g' \
                                    % (POL, THRESHOLD, record.nb_saccades, z_tilde, z_final, z_ref, toc - tic)   
                cpt_TRIALS  += NB_TRIALS

            dict_records[POL][THRESHOLD] = records
            print '\n'
            print 'Nb trials :', cpt_TRIALS
            #affiche_records(records)
            print '\n'
            np.save(file_name, dict_records)
else:
    dict_records = np.load(file_name)
        

('FEP-predictive', 1, 0)
Policy : FEP-predictive, threshold : 1, 1 saccades, initial : 3, final : 7, classe : 7, elapsed time : 3.56676


KeyboardInterrupt: 

In [71]:
print '\rPolicy : %s, threshold : %g, %d saccades, initial : %d, final : %d, classe : %d, elapsed time : %g' \
                                    % (POL, THRESHOLD, record.nb_saccades, z_tilde, z_final, z_ref, toc - tic)   

TypeError: %d format: a number is required, not numpy.ndarray

In [92]:
plt.plot(x_test)
plt.plot(x_noise)
print np.mean(x_test), np.mean(x_noise)
plt.plot(np.random.binomial(1,x_noise))

0.0923069 0.0923069350573


In [64]:
np.where(np.random.multinomial(1, np.ones(28)/28) == 1)[0][0]

23

In [55]:
np.log(1e-16)

-36.841361487904734